In [1]:
import mlflow

mlflow.set_tracking_uri(uri="http://localhost:8080")

In [2]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

/opt/miniconda3/envs/bgg/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [5]:
mlflow.set_experiment(experiment_name="MLflow quickstart")

2025/04/01 13:43:23 INFO mlflow.tracking.fluent: Experiment with name 'MLflow quickstart' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/212835036285219397', creation_time=1743507803425, experiment_id='212835036285219397', last_update_time=1743507803425, lifecycle_stage='active', name='MLflow quickstart', tags={}>

In [6]:
with mlflow.start_run():
    mlflow.log_params(params)

    mlflow.log_metric("accuracy", accuracy)

    mlflow.set_tag("Training info", "Basic LR model for iris data")

    signature = infer_signature(X_train, lr.predict(X_train))

    model_info = mlflow.sklearn.log_model(
        sk_model=lr, 
        artifact_path="iris_model", 
        signature=signature, 
        input_example=X_train, 
        registered_model_name="tracking-quickstart"
    )

Successfully registered model 'tracking-quickstart'.
2025/04/01 13:46:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1


🏃 View run blushing-boar-494 at: http://localhost:8080/#/experiments/212835036285219397/runs/17de8b37887b4f22b5dc53385fd8c381
🧪 View experiment at: http://localhost:8080/#/experiments/212835036285219397


Created version '1' of model 'tracking-quickstart'.


In [12]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
